In [2]:
import json
import pandas as pd
import urllib
from urllib.request import urlopen
import shapely
from shapely.geometry import Polygon
import geopandas
import laspy
import rasterio
from rasterio.plot import show
from rasterio import mask
import folium
import fiona

In [2]:
DATASET_PATH = "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/"
JSON_PATH = "../data/data_usgs.json"
        
def get_data(dataset_path:str,json_path:str):
    geo_dict = {'Region':[],
                'File_path':[],
                'Bounds':[],
                'WKT':[]
                }
    with open('../filename.txt') as filenames:
        for f in filenames:
            f = f.strip('\n')
            geo_dict['Region'].append(f)
            file = dataset_path + f +"ept.json"
            geo_dict['File_path'].append(file)
            try:
                meta = json.load(urlopen(file))
                wkt = meta["srs"]["wkt"]
                bounds = meta["bounds"]
            except:
                bounds = None                            
            geo_dict['Bounds'].append(bounds)    
            geo_dict['WKT'].append(wkt)
            
    with open(json_path,"w") as json_file:
        json.dump(geo_dict,json_file)                 

if __name__ == "__main__":
    get_data(DATASET_PATH,JSON_PATH)

A bound is an array of 6 numbers
[xmin, ymin,zmin,xmax,ymax,zmax]
We want to retain xmin,ymin,xmax,ymax

In [3]:
df = pd.read_json("../geosome/data/data_usgs.json")
df.head()

,Region,File_path,Bounds,WKT
0,AK_BrooksCamp_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-17347360, 8065364, -12414, -17321558, 809116...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
1,AK_Coastal_2009/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15730544, 10937407, -19027, -15691854, 10976...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
2,AK_Fairbanks-NSBorough_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16471700, 9519129, -45314, -16381190, 960963...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
3,AK_Juneau_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15014449, 8012267, -35030, -14943073, 808364...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
4,AK_Kenai_2008/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16906356, 8303726, -166851, -16570284, 86397...","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."


In [4]:
#Drop the z min and z max values
for i in df['Bounds']:
    del i[2]
for i in df['Bounds']:
    del i[4]

In [5]:
df.sample(5)

,Region,File_path,Bounds,WKT
121,CA_Scripps-Dec_2002/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-13121128, 3873238, -13036268, 3958098]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
846,USGS_LPC_IL_District4_Tazewell_2014_LAS_2016/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-10014350, 4905136, -9936068, 4983418]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
321,KS_SalineCo_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-10905618, 4653591, -10834300, 4724909]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
272,IN_Statewide_Opt1_B2_2017/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-9615730, 4820128, -9439430, 4996428]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."
1373,USGS_LPC_TX_Panhandle_B11_2017_LAS_2019/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-11356487, 4225403, -11130793, 4451097]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ..."


In [6]:
polygon = []
for b in df.Bounds:
    MINX,MINY,MAXX,MAXY = [b[0],b[1],b[2],b[3]]
    poly = Polygon(((MINX, MINY), (MINX, MAXY), (MAXX, MAXY), (MAXX, MINY), (MINX, MINY)))
    polygon.append(poly)

In [7]:
df['geometry'] = polygon
df.head()

,Region,File_path,Bounds,WKT,geometry
0,AK_BrooksCamp_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-17347360, 8065364, -17321558, 8091166]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-17347360 8065364, -17347360 8091166..."
1,AK_Coastal_2009/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15730544, 10937407, -15691854, 10976097]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-15730544 10937407, -15730544 109760..."
2,AK_Fairbanks-NSBorough_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16471700, 9519129, -16381190, 9609639]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-16471700 9519129, -16471700 9609639..."
3,AK_Juneau_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15014449, 8012267, -14943073, 8083643]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-15014449 8012267, -15014449 8083643..."
4,AK_Kenai_2008/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16906356, 8303726, -16570284, 8639798]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-16906356 8303726, -16906356 8639798..."


In [58]:
gdf = geopandas.GeoDataFrame(df)
gdf.head()

,Region,File_path,Bounds,WKT,geometry,area,centroid,envelope
0,AK_BrooksCamp_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-17347360, 8065364, -17321558, 8091166]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-17347360.000 8065364.000, -17347360...",6.657432e+08,POINT (-17334459.000 8078265.000),"POLYGON ((-17347360.000 8065364.000, -17321558..."
1,AK_Coastal_2009/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15730544, 10937407, -15691854, 10976097]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-15730544.000 10937407.000, -1573054...",1.496916e+09,POINT (-15711199.000 10956752.000),"POLYGON ((-15730544.000 10937407.000, -1569185..."
2,AK_Fairbanks-NSBorough_2010/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16471700, 9519129, -16381190, 9609639]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-16471700.000 9519129.000, -16471700...",8.192060e+09,POINT (-16426445.000 9564384.000),"POLYGON ((-16471700.000 9519129.000, -16381190..."
3,AK_Juneau_2012/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-15014449, 8012267, -14943073, 8083643]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-15014449.000 8012267.000, -15014449...",5.094533e+09,POINT (-14978761.000 8047955.000),"POLYGON ((-15014449.000 8012267.000, -14943073..."
4,AK_Kenai_2008/,https://s3-us-west-2.amazonaws.com/usgs-lidar-...,"[-16906356, 8303726, -16570284, 8639798]","PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-16906356.000 8303726.000, -16906356...",1.129444e+11,POINT (-16738320.000 8471762.000),"POLYGON ((-16906356.000 8303726.000, -16570284..."


In [59]:
gdf['area'] = gdf.area
# Returns a GeoSeries of points for each geometric centroid.
gdf['centroid'] = gdf.centroid
# Returns a GeoSeries of geometries representing the point or smallest rectangular
# polygon (with sides parallel to the coordinate axes) that contains each object.
gdf['envelope'] = gdf.envelope

In [60]:
gdf = gdf.drop(['File_path','Bounds'],axis=1)
gdf.sample()

,Region,WKT,geometry,area,centroid,envelope
264,IN_Statewide-StJosephCo_2011/,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-9633542.000 5073893.000, -9633542.0...",3.057648e+09,POINT (-9605894.000 5101541.000),"POLYGON ((-9633542.000 5073893.000, -9578246.0..."


In [61]:
gdf.crs = "EPSG:4326"
print(gdf.crs)

EPSG:4326


In [62]:
gdf.sample()

,Region,WKT,geometry,area,centroid,envelope
429,NC_BrunswickCo_2014/,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-8755722.000 4001331.000, -8755722.0...",6.558408e+09,POINT (-8715230.000 4041823.000),"POLYGON ((-8755722.000 4001331.000, -8674738.0..."


In [63]:
m = folium.Map(location=[37.0902,-95.7129], zoom_start=10, tiles='CartoDB positron')
m

In [64]:
gdf['lat'] = gdf.centroid.y
gdf['lon'] = gdf.centroid.x
gdf.head()

C:\Users\user\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
C:\Users\user\Downloads\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


,Region,WKT,geometry,area,centroid,envelope,lat,lon
0,AK_BrooksCamp_2012/,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-17347360.000 8065364.000, -17347360...",6.657432e+08,POINT (-17334459.000 8078265.000),"POLYGON ((-17347360.000 8065364.000, -17321558...",8078265.0,-17334459.0
1,AK_Coastal_2009/,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-15730544.000 10937407.000, -1573054...",1.496916e+09,POINT (-15711199.000 10956752.000),"POLYGON ((-15730544.000 10937407.000, -1569185...",10956752.0,-15711199.0
2,AK_Fairbanks-NSBorough_2010/,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-16471700.000 9519129.000, -16471700...",8.192060e+09,POINT (-16426445.000 9564384.000),"POLYGON ((-16471700.000 9519129.000, -16381190...",9564384.0,-16426445.0
3,AK_Juneau_2012/,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-15014449.000 8012267.000, -15014449...",5.094533e+09,POINT (-14978761.000 8047955.000),"POLYGON ((-15014449.000 8012267.000, -14943073...",8047955.0,-14978761.0
4,AK_Kenai_2008/,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-16906356.000 8303726.000, -16906356...",1.129444e+11,POINT (-16738320.000 8471762.000),"POLYGON ((-16906356.000 8303726.000, -16570284...",8471762.0,-16738320.0


In [65]:
region_list = []
years = []
states = []
for r in gdf.Region:
    r = r.strip("/")
    year = r.split('_')[-1]
    state = r.split('_')[1]
    region_list.append(r)
    years.append(year)
    states.append(state)

In [66]:
usgs_years = set(years)
print(usgs_years)

{'2016', '2013', '2003', '2014', '2005', '2008', '2009', '1', '2020', '2012', '2006-2008', '2017', '2006', '2019', '2002', 'NewYorkCity', '2015', '2004', '2018', '2011', '2010', 'Statewide-HarrisonCo-2011', '2012-2014', '2007', 'FullState'}


In [67]:
gdf['Region'] = region_list
gdf['State'] = states
gdf['Year'] = years
gdf.drop(['envelope','lat','lon'],axis=1,inplace=True)
gdf.head()

,Region,WKT,geometry,area,centroid,State,Year
0,AK_BrooksCamp_2012,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-17347360.000 8065364.000, -17347360...",6.657432e+08,POINT (-17334459.000 8078265.000),BrooksCamp,2012
1,AK_Coastal_2009,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-15730544.000 10937407.000, -1573054...",1.496916e+09,POINT (-15711199.000 10956752.000),Coastal,2009
2,AK_Fairbanks-NSBorough_2010,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-16471700.000 9519129.000, -16471700...",8.192060e+09,POINT (-16426445.000 9564384.000),Fairbanks-NSBorough,2010
3,AK_Juneau_2012,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-15014449.000 8012267.000, -15014449...",5.094533e+09,POINT (-14978761.000 8047955.000),Juneau,2012
4,AK_Kenai_2008,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-16906356.000 8303726.000, -16906356...",1.129444e+11,POINT (-16738320.000 8471762.000),Kenai,2008


In [68]:
options = ['NewYorkCity','Statewide-HarrisonCo-2011','FullState','1']
gdf.loc[gdf['Year'].isin(options)]

,Region,WKT,geometry,area,centroid,State,Year
227,IA_FullState,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-10758075.000 4793202.000, -10758075...",5.240920e+11,POINT (-10396104.000 5155173.000),FullState,FullState
255,IN_Statewide-HarrisonCo-2011,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-9619086.000 4572539.000, -9619086.0...",4.385883e+09,POINT (-9585973.000 4605652.000),Statewide-HarrisonCo-2011,Statewide-HarrisonCo-2011
333,KY_FullState,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-10309997.000 3793753.000, -10309997...",2.322771e+12,POINT (-9547965.000 4555785.000),FullState,FullState
373,MN_FullState,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-10913529.000 5383547.000, -10913529...",9.177065e+11,POINT (-10434544.000 5862532.000),FullState,FullState
519,NY_NewYorkCity,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-8267099.000 4937860.000, -8267099.0...",3.966480e+09,POINT (-8235609.000 4969350.000),NewYorkCity,NewYorkCity
751,USGS_LPC_CA_SanDiego_2015_C17_1,"PROJCS[""WGS 84 / Pseudo-Mercator"",GEOGCS[""WGS ...","POLYGON ((-13058061.000 3836206.000, -13058061...",1.360022e+10,POINT (-12999751.000 3894516.000),LPC,1


In [56]:
gdf = gdf.drop(gdf.index[[227,255,333,373,519,751]])